In [1]:
from random import seed, randint

seed(23)

import simpy


class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))
        self.bat_ctrl_proc = env.process(self.bat_ctrl(env))
        self.bat_ctrl_reactivate = env.event()

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1–6 hours
            print("Start parking at", env.now)
            self.bat_ctrl_reactivate.succeed()  # "reactivate"
            self.bat_ctrl_reactivate = env.event()
            yield env.timeout(randint(60, 360))
            print("Stop parking at", env.now)

    def bat_ctrl(self, env):
        while True:
            print("Bat. ctrl. passivating at", env.now)
            yield self.bat_ctrl_reactivate  # "passivate"
            print("Bat. ctrl. reactivated at", env.now)

            # Intelligent charging behavior here …
            yield env.timeout(randint(30, 90))


env = simpy.Environment()
ev = EV(env)
env.run(until=150)


Bat. ctrl. passivating at 0
Start parking at 29
Bat. ctrl. reactivated at 29
Bat. ctrl. passivating at 60
Stop parking at 131


In [2]:
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1–6 hours
            print("Start parking at", env.now)
            charging = env.process(self.bat_ctrl(env))
            parking = env.timeout(randint(60, 360))
            yield charging & parking
            print("Stop parking at", env.now)

    def bat_ctrl(self, env):
        print("Bat. ctrl. started at", env.now)
        # Intelligent charging behavior here …
        yield env.timeout(randint(30, 90))
        print("Bat. ctrl. done at", env.now)


env = simpy.Environment()
ev = EV(env)
env.run(until=310)


Start parking at 29
Bat. ctrl. started at 29
Bat. ctrl. done at 83
Stop parking at 305


In [3]:
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1 hour
            print("Start parking at", env.now)
            charging = env.process(self.bat_ctrl(env))
            parking = env.timeout(60)
            yield charging | parking
            if not charging.triggered:
                # Interrupt charging if not already done.
                charging.interrupt("Need to go!")
            print("Stop parking at", env.now)

    def bat_ctrl(self, env):
        print("Bat. ctrl. started at", env.now)
        try:
            yield env.timeout(randint(60, 90))
            print("Bat. ctrl. done at", env.now)
        except simpy.Interrupt as i:
            # Onoes! Got interrupted before the charging was done.
            print("Bat. ctrl. interrupted at", env.now, "msg:", i.cause)


env = simpy.Environment()
ev = EV(env)
env.run(until=100)


Start parking at 31
Bat. ctrl. started at 31
Stop parking at 91
Bat. ctrl. interrupted at 91 msg: Need to go!
